In [1]:
import pandas as pd
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Embedding
from tensorflow.keras.optimizers import Adam
from transformers import TFBertModel

/home/bendico765/PycharmProjects/HLT-project/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-24 16:15:36.143634: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-24 16:15:36.173280: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-24 16:15:36.174175: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-24 16:15:36.943503: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv("subtaskA_train.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1842 entries, 0 to 1841
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Id              1842 non-null   int64 
 1   comment_text    1842 non-null   object
 2   conspiratorial  1842 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 43.3+ KB


In [4]:
df = df[:50]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Id              50 non-null     int64 
 1   comment_text    50 non-null     object
 2   conspiratorial  50 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.3+ KB


## Costruzione di train e test set

In [6]:
X = df["comment_text"]
y = df["conspiratorial"]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.2, random_state = 10)

## Tokenizzazione con autotoken

In [8]:
from transformers import AutoTokenizer

def tokenize(X):
    X = tokenizer(
        text = list(X),
        add_special_tokens = True,
        max_length = 100,
        truncation = True,
        padding = 'max_length',
        return_tensors = 'tf',
        return_token_type_ids = False,
        return_attention_mask = True,
        verbose = True
        )
    return X

tokenizer = AutoTokenizer.from_pretrained('dbmdz/bert-base-italian-uncased')

In [9]:
X_train_tokens = tokenize(X_train)
X_test_tokens = tokenize(X_test)

## Creazione modello

In [10]:
length = 100
def get_model():
    dropout_rate = 0.2
    input_ids = Input(shape = (length,), dtype = tf.int32, name = 'input_ids')
    input_mask = Input(shape = (length,), dtype = tf.int32, name = 'input_mask')
    embeddings = bert([input_ids, input_mask])[1] #pooler output
    print(embeddings)
    out = Dropout(0.2)(embeddings)
    #64 units dense layer
    out = Dense(64,activation = 'relu')(out)
    out = Dropout(0.2)(out)
    y = Dense(1,activation = 'sigmoid')(out)
    model = Model(inputs=[input_ids, input_mask], outputs=y)
    model.layers[2].trainable = True
    #define optimizer
    optimizer = Adam(learning_rate=1e-05, epsilon=1e-08,clipnorm=1.0)
    #complile the model
    model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = 'accuracy')
    return model

In [11]:
bert = TFBertModel.from_pretrained('dbmdz/bert-base-italian-uncased')

Some layers from the model checkpoint at dbmdz/bert-base-italian-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at dbmdz/bert-base-italian-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [12]:
model = get_model()

KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='tf_bert_model/bert/pooler/dense/Tanh:0', description="created by layer 'tf_bert_model'")


In [13]:
from keras.callbacks import EarlyStopping

In [14]:
history = model.fit(
    x = {
        'input_ids': X_train_tokens['input_ids'],
        'input_mask':X_train_tokens['attention_mask']
    }, 
    y = y_train, 
    epochs=3, 
    validation_split = 0.2, 
    batch_size = 64, 
    callbacks= [ 
        EarlyStopping( monitor='val_accuracy' ,mode='max', patience=3,verbose=False,restore_best_weights=True)
    ]
)

Epoch 1/3
1/1 [==============================] - 35s 35s/step - loss: 0.7071 - accuracy: 0.5312 - val_loss: 0.7347 - val_accuracy: 0.2500
Epoch 2/3
1/1 [==============================] - 12s 12s/step - loss: 0.6909 - accuracy: 0.5938 - val_loss: 0.7297 - val_accuracy: 0.2500
Epoch 3/3
1/1 [==============================] - 12s 12s/step - loss: 0.7231 - accuracy: 0.4062 - val_loss: 0.7108 - val_accuracy: 0.3750
